In [2]:
pwd

'E:\\MLOps portfolio\\iris-flower-mlops\\notebooks'

In [3]:
import os 
os.chdir('E:\\MLOps portfolio\\iris-flower-mlops')

In [11]:
from src.get_data import read_params
import argparse
import mlflow
from mlflow.tracking import MlflowClient
from pprint import pprint
import joblib
import os



def log_production_model(config_path):
    config = read_params(config_path)
    
    
    mlflow_config = config["mlflow_config"] 
    

    model_name = mlflow_config["registered_model_name"]


    remote_server_uri = mlflow_config["remote_server_uri"]

    mlflow.set_tracking_uri(remote_server_uri)
    
    
    runs = mlflow.search_runs(experiment_ids='1')
    lowest = runs["metrics.mae"].sort_values(ascending=True)[0]
    lowest_run_id = runs[runs["metrics.mae"] == lowest]["run_id"][0]
    

    client = MlflowClient()
    for mv in client.search_model_versions(f"name='{model_name}'"):
        mv = dict(mv)
        pprint(mv)
        
        if mv["run_id"] == lowest_run_id:
            current_version = mv["version"]
            logged_model = mv["source"]
            pprint(mv, indent=4)
            client.transition_model_version_stage(
                name=model_name,
                version=current_version,
                stage="Production"
            )
        else:
            current_version = mv["version"]
            client.transition_model_version_stage(
                name=model_name,
                version=current_version,
                stage="Staging"
            )
log_production_model('params.yaml')

{'creation_timestamp': 1655460883204,
 'current_stage': 'Staging',
 'description': '',
 'last_updated_timestamp': 1655467338213,
 'name': 'RandomForestClassifier',
 'run_id': '8e3b8dd4af5d402d9d7a3cfb091c1576',
 'run_link': '',
 'source': './artifacts/1/8e3b8dd4af5d402d9d7a3cfb091c1576/artifacts/model',
 'status': 'READY',
 'status_message': '',
 'tags': {},
 'user_id': '',
 'version': '1'}
{'creation_timestamp': 1655460976457,
 'current_stage': 'Staging',
 'description': '',
 'last_updated_timestamp': 1655467338340,
 'name': 'RandomForestClassifier',
 'run_id': '34235075323d456ea40213840adaf0c1',
 'run_link': '',
 'source': './artifacts/1/34235075323d456ea40213840adaf0c1/artifacts/model',
 'status': 'READY',
 'status_message': '',
 'tags': {},
 'user_id': '',
 'version': '2'}
{'creation_timestamp': 1655461107614,
 'current_stage': 'Staging',
 'description': '',
 'last_updated_timestamp': 1655467338451,
 'name': 'RandomForestClassifier',
 'run_id': 'c6d76eb88ed449f08d8336e4e4c7f365',
 